In [1]:
import os
from pathlib import Path
from ultralytics import YOLO
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
model = YOLO('runs/detect/rtdetr_x_afdet_1/weights/best.pt')  # Replace with your model path

In [3]:
unlabeled_folder = 'Dataset/Aircraft_Fuselage_DET2023/unlabel_aircraft_fuselage'
output_dataset_folder = 'Dataset/pseudo_labeled_dataset'

In [4]:
# Create output directory structure
os.makedirs(os.path.join(output_dataset_folder, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dataset_folder, 'labels', 'train'), exist_ok=True)


In [5]:
# Get all image files
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
image_files = [f for f in os.listdir(unlabeled_folder) 
               if os.path.splitext(f)[1].lower() in image_extensions]

print(f"Found {len(image_files)} images to process...")

Found 5212 images to process...


In [6]:
image_file = '6593.jpg'  # Example: process the first image
image_path = os.path.join(unlabeled_folder, image_file)
    
# Run prediction
results = model.predict(image_path, conf=0.2)  # Adjust confidence threshold as needed
result = results[0]
result
    


image 1/1 /media/kaizen/T7/Project/final_project/Project/Dataset/Aircraft_Fuselage_DET2023/unlabel_aircraft_fuselage/6593.jpg: 640x640 2 paint_peels, 1 scratch, 45.6ms
Speed: 1.1ms preprocess, 45.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'paint_peel', 1: 'rivet_damage', 2: 'rust', 3: 'scratch'}
obb: None
orig_img: array([[[164, 169, 178],
        [157, 162, 171],
        [152, 157, 166],
        ...,
        [146, 152, 157],
        [152, 158, 163],
        [156, 162, 167]],

       [[163, 168, 177],
        [156, 161, 170],
        [151, 156, 165],
        ...,
        [147, 153, 158],
        [152, 158, 163],
        [156, 162, 167]],

       [[162, 167, 176],
        [156, 161, 170],
        [151, 156, 165],
        ...,
        [149, 155, 160],
        [153, 159, 164],
        [156, 162, 167]],

       ...,

       [[173, 178, 179],
        [172, 177, 178],
        [172, 177, 178],
        ...,
        [168, 173, 176],
        [166, 171, 174],
        [166, 171, 174]],

       [[175, 180, 181],
        [174, 179, 180],
        [174, 179, 180],
        ...,
        [166, 1

In [7]:
# Process each image
for idx, image_file in enumerate(image_files):
    image_path = os.path.join(unlabeled_folder, image_file)
    
    # Run prediction
    results = model.predict(image_path, conf=0.4)  # Adjust confidence threshold as needed
    result = results[0]
    
    # Get image dimensions
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]
    
    # Create YOLO label file
    label_filename = os.path.splitext(image_file)[0] + '.txt'
    conf_label_filename = os.path.splitext(image_file)[0] + "_conf" + '.txt'
    label_path = os.path.join(output_dataset_folder, 'labels', 'train', label_filename)
    conf_label_path = os.path.join(output_dataset_folder, 'labels', 'train', conf_label_filename)
    
    # Write predictions in YOLO format (class_id x_center y_center width height - normalized)
    with open(label_path, 'w') as f:
        with open(conf_label_path, 'w') as conf_f:
            for box in result.boxes:
                class_id = int(box.cls[0])
                x1, y1, x2, y2 = box.xyxy[0]
                
                # Convert to YOLO format (normalized coordinates)
                x_center = ((x1 + x2) / 2) / img_width
                y_center = ((y1 + y2) / 2) / img_height
                width = (x2 - x1) / img_width
                height = (y2 - y1) / img_height
                
                confidence = float(box.conf[0])
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
                conf_f.write(f"{confidence:.6f}\n")
    
    # Copy image to output folder
    output_image_path = os.path.join(output_dataset_folder, 'images', 'train', image_file)
    img_output = cv2.imread(image_path)
    cv2.imwrite(output_image_path, img_output)
    
    if (idx + 1) % 10 == 0:
        print(f"Processed {idx + 1}/{len(image_files)} images...")

print("Pseudo-labeling complete!")
print(f"Dataset saved to: {output_dataset_folder}")


image 1/1 /media/kaizen/T7/Project/final_project/Project/Dataset/Aircraft_Fuselage_DET2023/unlabel_aircraft_fuselage/6099.jpg: 640x640 1 rivet_damage, 45.9ms
Speed: 1.2ms preprocess, 45.9ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /media/kaizen/T7/Project/final_project/Project/Dataset/Aircraft_Fuselage_DET2023/unlabel_aircraft_fuselage/6262.jpg: 640x640 (no detections), 46.5ms
Speed: 1.4ms preprocess, 46.5ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /media/kaizen/T7/Project/final_project/Project/Dataset/Aircraft_Fuselage_DET2023/unlabel_aircraft_fuselage/7790.jpg: 640x640 14 paint_peels, 50.4ms
Speed: 1.2ms preprocess, 50.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /media/kaizen/T7/Project/final_project/Project/Dataset/Aircraft_Fuselage_DET2023/unlabel_aircraft_fuselage/1974.jpg: 640x640 (no detections), 46.7ms
Speed: 1.7ms preprocess, 46.7ms inference, 0.2ms postprocess per image 

In [12]:
# # Create visualization folder
# viz_folder = os.path.join(output_dataset_folder, 'visualizations')
# os.makedirs(viz_folder, exist_ok=True)

# # Process each image with visualizations
# for idx, image_file in enumerate(image_files):
#     image_path = os.path.join(unlabeled_folder, image_file)
#     img = cv2.imread(image_path)
#     img_height, img_width = img.shape[:2]
    
#     # Read labels and confidence scores
#     label_filename = os.path.splitext(image_file)[0] + '.txt'
#     conf_label_filename = os.path.splitext(image_file)[0] + "_conf" + '.txt'
#     label_path = os.path.join(output_dataset_folder, 'labels', 'train', label_filename)
#     conf_label_path = os.path.join(output_dataset_folder, 'labels', 'train', conf_label_filename)
    
#     # Draw bounding boxes on image
#     if os.path.exists(label_path):
#         with open(label_path, 'r') as f:
#             labels = f.readlines()
#         with open(conf_label_path, 'r') as conf_f:
#             confidences = conf_f.readlines()
        
#         for label_line, conf_line in zip(labels, confidences):
#             parts = label_line.strip().split()
#             conf_score = float(conf_line.strip())
            
#             class_id = int(parts[0])
#             x_center = float(parts[1]) * img_width
#             y_center = float(parts[2]) * img_height
#             width = float(parts[3]) * img_width
#             height = float(parts[4]) * img_height
            
#             x1 = int(x_center - width / 2)
#             y1 = int(y_center - height / 2)
#             x2 = int(x_center + width / 2)
#             y2 = int(y_center + height / 2)
            
#             # Draw rectangle
#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
#             # Put confidence score text
#             text = f"Class {class_id}: {conf_score:.2f}"
#             cv2.putText(img, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 
#                        0.5, (0, 255, 0), 2)
    
#     # Save visualization
#     output_viz_path = os.path.join(viz_folder, image_file)
#     cv2.imwrite(output_viz_path, img)
    
#     if (idx + 1) % 10 == 0:
#         print(f"Visualized {idx + 1}/{len(image_files)} images...")

# print("Visualization complete!")
# print(f"Visualizations saved to: {viz_folder}")

[ WARN:0@6021.670] global loadsave.cpp:248 findDecoder imread_('Dataset/Aircraft_Fuselage_DET2023/unlabel_aircraft_fuselage/039.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'